### Simple RNN example with tensorflow.

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from tensorflow.contrib import rnn

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [18]:
df = pd.read_csv('/Users/u606941/VA Scripts/rnn_test.csv',low_memory=False)

#### Simple time series with spikes every 6 months.

In [19]:
df.head(10)

,series
0,200
1,42
2,42
3,42
4,41
5,41
6,202
7,41
8,42
9,39


#### Convert dataframe series into an array and then create X_train and Y_train by creating lag

In [20]:
arr = np.array(df)
y_train = arr[1:].copy()
X_train = arr[:-1].copy()

In [21]:
np.c_[X_train,y_train]

array([[200,  42],
       [ 42,  42],
       [ 42,  42],
       [ 42,  41],
       [ 41,  41],
       [ 41, 202],
       [202,  41],
       [ 41,  42],
       [ 42,  39],
       [ 39,  41],
       [ 41,  40],
       [ 40, 201],
       [201,  44],
       [ 44,  42],
       [ 42,  45],
       [ 45,  42],
       [ 42,  43],
       [ 43, 203],
       [203,  43],
       [ 43,  43],
       [ 43,  44],
       [ 44,  42],
       [ 42,  45]])

#### We have 23 steps with single feature input and single output.

In [22]:
n_steps = 23
n_inputs = 1
n_neurons = 10
n_outputs = 1

#### declare placeholders and declare a Basic RNN cell

In [23]:
reset_graph()

X = tf.placeholder(tf.float32, [None,n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, \
                                                                activation = tf.nn.relu), output_size = n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [24]:
learning_rate = 0.001
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [25]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

#### Build the model

In [26]:
for i in range(500):
    X_batch = X_train.reshape(1, 23, 1)
    y_batch = y_train.reshape(1, 23, 1)
    sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
    mse = loss.eval(feed_dict = {X:X_batch, y:y_batch})
    print(mse)

5034.41
4998.32
4963.48
4929.55
4896.09
4863.46
4834.74
4817.87
4807.08
4798.63
4789.47
4779.92
4768.84
4756.65
4743.15
4728.41
4712.53
4696.44
4680.56
4664.89
4649.31
4633.29
4617.87
4602.29
4587.03
4571.66
4556.19
4540.29
4523.93
4507.16
4490.05
4472.5
4454.51
4435.9
4416.83
4397.45
4378.0
4358.24
4338.31
4317.97
4297.07
4275.6
4254.13
4232.27
4210.03
4187.26
4163.94
4140.06
4115.59
4090.52
4064.83
4039.17
4013.79
3987.92
3961.58
3934.75
3907.44
3879.65
3851.35
3822.55
3793.22
3764.68
3739.47
3713.86
3687.38
3660.11
3632.16
3603.6
3574.52
3545.2
3515.93
3486.39
3456.63
3426.88
3399.33
3372.01
3343.31
3313.51
3284.21
3256.32
3228.59
3200.98
3173.49
3146.1
3118.5
3090.42
3061.78
3032.58
3004.31
2975.8
2947.27
2918.44
2889.34
2860.37
2831.49
2807.14
2781.94
2756.14
2729.49
2703.45
2677.14
2651.01
2625.36
2599.07
2574.03
2549.04
2523.95
2498.74
2473.36
2449.33
2424.63
2400.15
2375.84
2351.25
2325.61
2300.03
2275.85
2252.33
2228.04
2202.68
2176.98
2152.21
2127.28
2102.14
2076.25
2049.98
2

#### Use output values (y_batch) as input to predict next 10 values in a loop

In [27]:
pred_list = []
for i in range(10):
    X_new = y_batch
    y_pred = sess.run(outputs, feed_dict={X:X_new})
    pred_list.append(y_pred[:,22:,:])
    y_batch = np.append(y_batch[:,1:,:],y_pred[:,22:,:],axis=1)

In [28]:
pred_list

[array([[[ 205.42623901]]], dtype=float32),
 array([[[ 43.55230331]]], dtype=float32),
 array([[[ 44.43833542]]], dtype=float32),
 array([[[ 43.61295319]]], dtype=float32),
 array([[[ 44.867733]]], dtype=float32),
 array([[[ 44.47499847]]], dtype=float32),
 array([[[ 207.99891663]]], dtype=float32),
 array([[[ 44.42189407]]], dtype=float32),
 array([[[ 44.14424515]]], dtype=float32),
 array([[[ 43.9930687]]], dtype=float32)]

In [29]:
sess.close()

#### these are good predictions with spike every 6 months like in our training data